# Analysis of 1/48 Deg global ocean run
Relevant links -

https://medium.com/pangeo/petabytes-of-ocean-data-part-1-nasa-ecco-data-portal-81e3c5e077be 

https://xmitgcm.readthedocs.io/en/latest/llcreader.html

https://pangeo-data.github.io/pangeo-datastore/master/ocean/llc4320.html#llc4320-grid

https://ecco.jpl.nasa.gov/products/latest/user-guide/

In [1]:
import xarray as xr
import dask.array as da
from dask.distributed import Client, LocalCluster
from xgcm import Grid
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from xmitgcm import llcreader
import intake
%matplotlib inline

In [5]:
lc = LocalCluster(n_workers=1)
client = Client(lc)
client

Client Scheduler: tcp://127.0.0.1:45223 Dashboard: http://127.0.0.1:35991/status,Cluster Workers: 1 Cores: 16 Memory: 541.57 GB


In [6]:
# Load grid data from web
cat = intake.Catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/llc4320.yaml")
dsgrid = cat["LLC4320_grid"].to_dask()
dsgrid = llcreader.faces_dataset_to_latlon(dsgrid, metric_vector_pairs=[('dxC', 'dyC'), ('dyG', 'dxG')])
#print(dsgrid)

FileNotFoundError: https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/llc4320.yaml

In [7]:
# Reads data on the web.  
model = llcreader.ECCOPortalLLC4320Model()
ds = model.get_dataset(varnames=['U', 'V', 'W'], k_levels=[1, 3, 5], type = 'latlon')
#print(ds)

KeyError: 'https://storage.googleapis.com/pangeo-ecco/llc/masks/llc_4320_masks.zarr/.zmetadata'

In [15]:
# define grid
ds = xr.merge([ds, dsgrid])
grid = Grid(ds, coords={'X': {'center': 'i', 'right': 'i_g'},
                        'Y': {'center': 'j', 'right': 'j_g'} }, periodic=['X']);

In [17]:
ds

<xarray.Dataset>
Dimensions:  (face: 13, i: 17280, i_g: 17280, j: 12960, j_g: 12960, k: 3, k_l: 3, k_p1: 4, k_u: 3, time: 9030)
Coordinates:
  * k_p1     (k_p1) int64 0 1 3 5
  * i        (i) int64 0 1 2 3 4 5 6 ... 17274 17275 17276 17277 17278 17279
  * k_u      (k_u) int64 1 3 5
  * k_l      (k_l) int64 1 3 5
  * time     (time) datetime64[ns] 2011-09-13 ... 2012-09-23T05:00:00
  * j        (j) int64 0 1 2 3 4 5 6 ... 12954 12955 12956 12957 12958 12959
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * j_g      (j_g) int64 0 1 2 3 4 5 6 ... 12954 12955 12956 12957 12958 12959
  * k        (k) int64 1 3 5
  * i_g      (i_g) int64 0 1 2 3 4 5 6 ... 17274 17275 17276 17277 17278 17279
    CS       (j, i) float32 dask.array<chunksize=(4320, 4320), meta=np.ndarray>
    Depth    (j, i) float32 dask.array<chunksize=(4320, 4320), meta=np.ndarray>
    PHrefC   float32 15.4017
    PHrefF   (k_p1) float32 dask.array<chunksize=(4,), meta=np.ndarray>
    SN       (j, i) float32 dask.arr

In [21]:
lon = ds.XC.values

In [24]:
lon.plot()

AttributeError: 'numpy.ndarray' object has no attribute 'plot'

In [27]:
ds.nbytes/1e12

72.817400846196

In [28]:
ds.close()